# `model_leaf` function

**A function `model_leaf` that takes inputs of:**

1. leaf trace coordinates
2. landmark coordinates
3. number of points to interpolate for the trace
4. number of landmarks (interpolated points within each lobe)
5. polynomial degree to model each landmark across leaf lobes
6. frames (or x values) to model across the leaf, apportioned to intervals between lobes by rounding

**The function returns:**

1. overall modeled x and y values. for both x and y values, this is a double list of lists structured first by frame, and the for each frame, modeled values for each landmark
2. interpolated trace lists for plotting
3. landmark lists that have been aligned to the trace (for plotting)

In [ ]:
import numpy as np # for text file data
from PIL import Image # for displaying images
from scipy.interpolate import interp1d # for interpolating points
import matplotlib.pyplot as plt # for plotting
%matplotlib inline

# first define a function to return equally spaced, interpolated points

def interpolation(x, y, number): 

    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular

# now define the model_leaf function

def model_leaf(trace_file, landmarks_file, num_points, num_land, deg, frames):

    #############################
    #############################
    #############################

    trace_xvals, trace_yvals = interpolation(trace_file[:,0],trace_file[:,1],num_points)

    #############################
    # find interpolated trace points that correspond to the landmarks
    #############################

    # find the corresponding trace coordinates for each landmark
    # record new landmark coord values
    # record landmark index values for trace values

    land_xvals = [] # list to store new landmark x vals
    land_yvals = [] # list to store new landmark y vals
    land_indices = [] # list to store index values

    for i in range(landmarks_file.shape[0]):

        landx = landmarks_file[i,0] # select current landmark
        landy = landmarks_file[i,1] 

        distances = [] # list to store distances of current landmark with each trace coord

        for j in range(len(trace_xvals)): # for each trace coord

            tracex = trace_xvals[j]
            tracey = trace_yvals[j]

            d = np.sqrt( (landx-tracex)**2 + (landy-tracey)**2 ) # find distance to each landmark

            distances.append(d) # append to distances

        min_val = np.min(distances) # find min distance value
        min_ind = distances.index(min_val) # find min index value among trace coords

        land_xvals.append(trace_xvals[min_ind]) # append new landmark coord and index vals
        land_yvals.append(trace_yvals[min_ind])
        land_indices.append(min_ind)

    #############################
    # create same number of interpolated landmarks for each lobe, using beginning, tip, and end as true landmarks
    #############################

    num_lobes = int((len(land_indices)-1)/2) # get number of lobes in the leaf

    lobe_xvals = []
    lobe_yvals = []

    for i in range(num_lobes): # for the number of lobes

        begin = i*2 # current lobe beginning index
        tip = i*2+1 # current lobe tip index
        end = i*2+2 # current lobe end index

        first_half_x = trace_xvals[land_indices[begin]:land_indices[tip]] # isolate first half of traced lobe
        first_half_y = trace_yvals[land_indices[begin]:land_indices[tip]]
        second_half_x = trace_xvals[land_indices[tip]:land_indices[end]]  # isolate second half of traced lobe
        second_half_y = trace_yvals[land_indices[tip]:land_indices[end]]

        first_landx, first_landy = interpolation(first_half_x, first_half_y, int(num_land/2)) # interpolate half number of landmarks to first half
        second_landx, second_landy = interpolation(second_half_x, second_half_y, int(num_land/2)) # interpolate half number of landmarks to second half

        lobex = list(first_landx) + list(second_landx) # combine xvals of first and second half of lobe
        lobey = list(first_landy) + list(second_landy) # combine yvals of first and second half of lobe

        lobe_xvals.append(lobex) # append a list with the interpolated coordinate values for each lobe
        lobe_yvals.append(lobey)

    lobex_arr = np.array(lobe_xvals) # turn list of lists of lobe values into arrays
    lobey_arr = np.array(lobe_yvals)

    #############################
    # for frame number, apportion frames approximately equally between intervals by rounding
    #############################

    num_lobes = int((len(land_indices)-1)/2) # get number of lobes in the leaf

    inv_num = np.linspace(0,frames,num_lobes) # calculate interval number (between lobe spaces)

    round_inv = [int(round(item, 0)) for item in inv_num] # rounds the above list to number of frames for each interval

    intervals = [] # a list of lists, for each interval, a list of the frames it is apportioned

    for r in range(len(round_inv)-1):

        inv_begin = round_inv[r]
        inv_end = round_inv[r+1]

        interval = []

        for n in range(inv_begin, inv_end):

            interval.append(n)

        intervals.append(interval)

    #############################
    # create double list of lists, for each frame, a list of all modeled values
    #############################

    overall_xvals = [] # a double list of lists, for each frame, a list of landmark values
    overall_yvals = [] # a double list of lists, for each frame, a list of landmark values

    for f in range(frames): # for each frame

        curr_frame_xvals = [] # a list of modeled values for each landmark for the current frame
        curr_frame_yvals = []

        for l in range(num_land):

            # for current landmark l, code below recalculates and apportions xvals across leaf
            # this code is repetitive, but puts frame number before calculating modeled values
            all_xvals = []
            for k in range(len(intervals)):
                curr_inv = intervals[k]
                inv_len = len(curr_inv)
                curr_xvals = np.linspace(lobex_arr[k,l],lobex_arr[k+1,l], inv_len)
                all_xvals.append(curr_xvals)
            flat_all_xvals = []
            for i in range(len(all_xvals)):
                curr_list = all_xvals[i]
                for j in range(len(curr_list)):
                    flat_all_xvals.append(curr_list[j])

            # isolate the current xval for the current frame, for the current landmark
            curr_xval = flat_all_xvals[f]

            # recalculates polynomial function for the current landmark l
            coef = np.polyfit(lobex_arr[:,l],lobey_arr[:,l],deg) # fit a polynomial for the current landmark
            polyfunc = np.poly1d(coef) # create polynomial function

            # for the current frame and landmark, find modeled value for the calculated xval
            modeled_val = polyfunc(curr_xval) 

            curr_frame_xvals.append(curr_xval) # append current x val
            curr_frame_yvals.append(modeled_val) # append current modeled val

        overall_xvals.append(curr_frame_xvals) # append list of xvals for current frame
        overall_yvals.append(curr_frame_yvals) # append list of yvals for current frame

    return overall_xvals, overall_yvals, trace_xvals, trace_yvals, land_xvals, land_yvals


# data for modeling four different *Capsella* leaves

**Data is from Figure 1A of:**

Iannetta PP, Begg G, Hawes C, Young M, Russell J, Squire GR. **Variation in Capsella (shepherd’s purse): an example of intraspecific functional diversity.** *Physiologia Plantarum*. 2007 Mar;129(3):542-54. [doi](https://doi.org/10.1111/j.1399-3054.2006.00833.x)

In [ ]:
# read in a poly line trace of the leaf outline
# read in landmarks of the beginning, tip, and end of each lobe
# four leaves (A-D), two sides each (upper and lower)

uA_trace = np.loadtxt("./data/upperA_trace.txt" ) # polyline trace
uA_land = np.loadtxt("./data/upperA_landmarks.txt" ) # lobe landmarks
lA_trace = np.loadtxt("./data/lowerA_trace.txt" ) # polyline trace
lA_land = np.loadtxt("./data/lowerA_landmarks.txt" ) # lobe landmarks

uB_trace = np.loadtxt("./data/upperB_trace.txt" ) # polyline trace
uB_land = np.loadtxt("./data/upperB_landmarks.txt" ) # lobe landmarks
lB_trace = np.loadtxt("./data/lowerB_trace.txt" ) # polyline trace
lB_land = np.loadtxt("./data/lowerB_landmarks.txt" ) # lobe landmarks

uC_trace = np.loadtxt("./data/upperC_trace.txt" ) # polyline trace
uC_land = np.loadtxt("./data/upperC_landmarks.txt" ) # lobe landmarks
lC_trace = np.loadtxt("./data/lowerC_trace.txt" ) # polyline trace
lC_land = np.loadtxt("./data/lowerC_landmarks.txt" ) # lobe landmarks

uD_trace = np.loadtxt("./data/upperD_trace.txt" ) # polyline trace
uD_land = np.loadtxt("./data/upperD_landmarks.txt" ) # lobe landmarks
lD_trace = np.loadtxt("./data/lowerD_trace.txt" ) # polyline trace
lD_land = np.loadtxt("./data/lowerD_landmarks.txt" ) # lobe landmarks

# set the parameters

num_points = 200 # number of trace points to interpolate
num_land = 20 # number of landmarks for each lobe, make it even number
deg = 2 # degrees of fitted polynomial
frames = 200 # number of frames across the leaf to model

# use function model_leaf to retrieve data

overUAX, overUAY, traceUAX, traceUAY, landUAX, landUAY = model_leaf(uA_trace, uA_land, num_points, num_land, deg, frames)
overLAX, overLAY, traceLAX, traceLAY, landLAX, landLAY = model_leaf(lA_trace, lA_land, num_points, num_land, deg, frames)

overUBX, overUBY, traceUBX, traceUBY, landUBX, landUBY = model_leaf(uB_trace, uB_land, num_points, num_land, deg, frames)
overLBX, overLBY, traceLBX, traceLBY, landLBX, landLBY = model_leaf(lB_trace, lB_land, num_points, num_land, deg, frames)

overUCX, overUCY, traceUCX, traceUCY, landUCX, landUCY = model_leaf(uC_trace, uC_land, num_points, num_land, deg, frames)
overLCX, overLCY, traceLCX, traceLCY, landLCX, landLCY = model_leaf(lC_trace, lC_land, num_points, num_land, deg, frames)

overUDX, overUDY, traceUDX, traceUDY, landUDX, landUDY = model_leaf(uD_trace, uD_land, num_points, num_land, deg, frames)
overLDX, overLDY, traceLDX, traceLDY, landLDX, landLDY = model_leaf(lD_trace, lD_land, num_points, num_land, deg, frames)


# visualize the results

In [ ]:
# read in photo file
# image file that we are using

photo_file = Image.open("./original_image/Iannetta_2007.jpg", 'r')

outline_c = "magenta"
outline_lw = 2

land_c = "orange"
land_s = 70

point_c = "cyan" # modeled point parameters
point_lw = 2

highlight_c = "yellow" # highlight point parameters
highlight_s = 70

for f in range(frames):

    fig = plt.figure(figsize=(10*0.75,14*0.75), facecolor="black")

    plt.imshow(photo_file, cmap="gist_gray")
    
    # plot traces and landmarks for upper and lower sides of each leaf

    plt.plot(traceUAX,traceUAY, c=outline_c, lw=outline_lw)
    plt.scatter(landUAX,landUAY, c=land_c, s=land_s)

    plt.plot(traceLAX,traceLAY, c=outline_c, lw=outline_lw)
    plt.scatter(landLAX,landLAY, c=land_c, s=land_s)

    plt.plot(traceUBX,traceUBY, c=outline_c, lw=outline_lw)
    plt.scatter(landUBX,landUBY, c=land_c, s=land_s)

    plt.plot(traceLBX,traceLBY, c=outline_c, lw=outline_lw)
    plt.scatter(landLBX,landLBY, c=land_c, s=land_s)

    plt.plot(traceUCX,traceUCY, c=outline_c, lw=outline_lw)
    plt.scatter(landUCX,landUCY, c=land_c, s=land_s)

    plt.plot(traceLCX,traceLCY, c=outline_c, lw=outline_lw)
    plt.scatter(landLCX,landLCY, c=land_c, s=land_s)

    plt.plot(traceUDX,traceUDY, c=outline_c, lw=outline_lw)
    plt.scatter(landUDX,landUDY, c=land_c, s=land_s)

    plt.plot(traceLDX,traceLDY, c=outline_c, lw=outline_lw)
    plt.scatter(landLDX,landLDY, c=land_c, s=land_s)

    # plot current frame of modeled values for upper and lower sides of each leaf

    plt.plot(overUAX[f], overUAY[f], c=point_c, lw=point_lw)
    plt.scatter(overUAX[f][0], overUAY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overUAX[f][10], overUAY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overUAX[f][19],  overUAY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overLAX[f],  overLAY[f], c=point_c, lw=point_lw)
    plt.scatter(overLAX[f][0], overLAY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overLAX[f][10], overLAY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overLAX[f][19],  overLAY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overUBX[f], overUBY[f], c=point_c, lw=point_lw)
    plt.scatter(overUBX[f][0], overUBY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overUBX[f][10], overUBY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overUBX[f][19],  overUBY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overLBX[f],  overLBY[f], c=point_c, lw=point_lw)
    plt.scatter(overLBX[f][0], overLBY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overLBX[f][10], overLBY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overLBX[f][19],  overLBY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overUCX[f], overUCY[f], c=point_c, lw=point_lw)
    plt.scatter(overUCX[f][0], overUCY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overUCX[f][10], overUCY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overUCX[f][19],  overUCY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overLCX[f],  overLCY[f], c=point_c, lw=point_lw)
    plt.scatter(overLCX[f][0], overLCY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overLCX[f][10], overLCY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overLCX[f][19],  overLCY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overUDX[f], overUDY[f], c=point_c, lw=point_lw)
    plt.scatter(overUDX[f][0], overUDY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overUDX[f][10], overUDY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overUDX[f][19],  overUDY[f][19], c=highlight_c, s=highlight_s)

    plt.plot(overLDX[f],  overLDY[f], c=point_c, lw=point_lw)
    plt.scatter(overLDX[f][0], overLDY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(overLDX[f][10], overLDY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(overLDX[f][19],  overLDY[f][19], c=highlight_c, s=highlight_s)
    
    plt.axis("off")
    
    filename = "./plots/frame"+str(f)+".jpg"
        
    plt.savefig(filename, facecolor=fig.get_facecolor())
    plt.close()